# Block 7 Übung 1: Orientierung in den Daten der MIMIC-Tabelle Chartevents

## Bibliotheken & Konfiguration

In [ ]:
packages <- c("readr", "dplyr")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/"

## MIMIC III-Basisdaten laden

In [ ]:
# Patientenstammdaten laden
mimic.patients.raw <- read_csv(paste(base_url, "mimic-iii-demo/PATIENTS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()))

# Behandlungsfälle laden
mimic.admissions.raw <- read_csv(paste(base_url, "mimic-iii-demo/ADMISSIONS.csv", sep=""),
                                 col_types = cols(  row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()))

# Intensivaufenthalte laden
mimic.icustays.raw <- read_csv(paste(base_url, "mimic-iii-demo/ICUSTAYS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), icustay_id = col_integer(), dbsource = col_character(), first_careunit = col_character(), last_careunit = col_character(), first_wardid = col_double(), last_wardid = col_double(), intime = col_datetime(format = ""), outtime = col_datetime(format = ""), los = col_double()))


## MIMIC III-Chartevents laden

In [ ]:
# Chartevents laden
mimic.chartevents.raw <- read_csv(paste(base_url, "mimic-iii-demo/CHARTEVENTS.csv", sep=""),
                                  col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), icustay_id = col_integer(), itemid = col_integer(), charttime = col_datetime(format = ""), storetime = col_datetime(format = ""), cgid = col_double(), value = col_character(), valuenum = col_double(), valueuom = col_character(), warning = col_double(), error = col_double(), resultstatus = col_character(), stopped = col_character()))
head(mimic.chartevents.raw)

### Bezeichner ergänzen

In [ ]:
# Bezeichner laden
mimic.d_items.raw <- read_csv(paste(base_url, "mimic-iii-demo/D_ITEMS.csv", sep=""),
                              col_types = cols(row_id = col_integer(), itemid = col_integer(), label = col_character(), abbreviation = col_character(), dbsource = col_character(), linksto = col_character(), category = col_character(), unitname = col_character(), param_type = col_character(), conceptid = col_character()))

# Bezeichner hinzufügen
mimic.chartevents.annotated <- mimic.chartevents.raw %>%
  inner_join(mimic.d_items.raw %>% select(itemid, category, label), by = "itemid")

head(mimic.chartevents.annotated)

## Aufgabe: Finden Sie häufig verwendete Items für den Blutdruck und Puls!

Hier können Sie das Wissen aus dem Abschnitt 4, Demo 2 (Daten abfragen mit Tidyverse) anwenden.
* zugehöriges Notebook: [teil-4-demo-02-abfragen-tidyverse.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-4-demo-02-abfragen-tidyverse.ipynb)

Filtern Sie die Rohdaten mit der `filter()`-Funktion nach Inhalten, die Sie für Blutdruck & Puls erwarten würden (z.B. Maßeinheiten in der Spalte `valueuom` oder Bezeichner in der Spalte `label`. Finden Sie häufig verwendete Attribute, indem Sie die Anzahl der Zeilen mit den `group_by()`- und `summarize()`-Funktionen aggregieren.

Tipps zur `filter()`-Funktion:
* Wenn Sie mehrere mit Komma getrennte Kriterien angeben, werden sie mit logisch "und" verknüpft
* Sie können Kriterien mit dem `|`-Operator auch als logisch "oder" kombinieren
* Sie können mit dem `%in%`-Operator auch nach mehreren verschiedenen Ausprägungen eines Textes suchen (z.B. `filter(farbe %in% c("rot", "blau"))`)
* Sie können auch nach Teilstrings suchen, indem Sie die Funktion `grepl("Suchbegriff", Spalte)` nutzen

Sonstige Tipps:
* Denken Sie daran, dass Sie bei der `summarize()`-Funktion die Option `.groups = "keep"` angeben müssen, wenn die Gruppierungsmerkmale beibehalten werden sollen. Sie erhalten sonst nur das aggregierte Ergebnis als Zahl, aber nicht die (meistens benötigten) Merkmale, nach denen zuvor gruppiert wurde.

In [ ]:
# Mögliche Ausprägungen für den Blutdruck finden
mimic.chartevents.annotated %>% filter(valueuom == "mmHg", grepl("systolic", tolower(label)) | grepl("diastolic", tolower(label))) %>%
    group_by(itemid, label, valueuom) %>%
    summarize(n = n(), .groups = "keep")

In [ ]:
# Mögliche Ausprägungen für den Puls finden
mimic.chartevents.annotated %>% filter(valueuom == "bpm") %>%
    group_by(itemid, label, valueuom) %>%
    summarize(n = n(), .groups = "keep")